# 5.1 Zbiór danych: Wstrząsy sejsmiczne
Detekcja anomalii jest problemem innym niż regresja czy klasyfikacja. Typowo nie mamy tutaj danych docelowych etykiet danych, ewentualnie etykiety są przypisane tylko części obserwacji. Ocena jakośc takiego modelu jest więc dość utrudniona. My rozpatrzymy zbiór, który zawiera w sobie informacje odnośnie wstrząsów sejsmicznych z kopalni, które mają już przypisane klasy, jednak nie będziemy ich używać w fazie uczenia, żeby być w stanie zobaaczyć jak poprawny jest nasz system.

In [1]:
from scipy.io import arff

In [2]:
import pandas as pd

In [3]:
data, description = arff.loadarff("../data/seismic-bumps.arff")
description

Dataset: seismic-bumps
	seismic's type is nominal, range is ('a', 'b', 'c', 'd')
	seismoacoustic's type is nominal, range is ('a', 'b', 'c', 'd')
	shift's type is nominal, range is ('W', 'N')
	genergy's type is numeric
	gpuls's type is numeric
	gdenergy's type is numeric
	gdpuls's type is numeric
	ghazard's type is nominal, range is ('a', 'b', 'c', 'd')
	nbumps's type is numeric
	nbumps2's type is numeric
	nbumps3's type is numeric
	nbumps4's type is numeric
	nbumps5's type is numeric
	nbumps6's type is numeric
	nbumps7's type is numeric
	nbumps89's type is numeric
	energy's type is numeric
	maxenergy's type is numeric
	class's type is nominal, range is ('1', '0')

In [4]:
seismic_bumps_df = pd.DataFrame(data)
seismic_bumps_df.sample(5)

,seismic,seismoacoustic,shift,genergy,gpuls,gdenergy,gdpuls,ghazard,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,nbumps6,nbumps7,nbumps89,energy,maxenergy,class
886,b'a',b'a',b'W',20090.0,74.0,-78.0,-62.0,b'a',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
882,b'a',b'a',b'W',125340.0,292.0,35.0,62.0,b'a',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
62,b'a',b'a',b'N',31540.0,121.0,-32.0,-30.0,b'a',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
915,b'a',b'a',b'N',12950.0,41.0,-65.0,-55.0,b'a',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'
229,b'a',b'a',b'W',1345240.0,3510.0,-17.0,16.0,b'a',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,b'0'


In [5]:
byte_columns = seismic_bumps_df.select_dtypes(["object"]).columns

In [6]:
seismic_bumps_df[byte_columns] = seismic_bumps_df[byte_columns].applymap(lambda x: x.decode("utf-8"))
seismic_bumps_df.sample(5)

/tmp/ipykernel_18471/1418440619.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  seismic_bumps_df[byte_columns] = seismic_bumps_df[byte_columns].applymap(lambda x: x.decode("utf-8"))


,seismic,seismoacoustic,shift,genergy,gpuls,gdenergy,gdpuls,ghazard,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,nbumps6,nbumps7,nbumps89,energy,maxenergy,class
2104,a,a,N,7080.0,260.0,-41.0,5.0,a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
677,a,a,N,11440.0,121.0,-68.0,-37.0,a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1868,a,b,N,8690.0,92.0,-12.0,-74.0,a,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4000.0,4000.0,0
249,a,b,N,154420.0,319.0,-46.0,-43.0,a,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3000.0,3000.0,0
2162,b,a,W,53660.0,627.0,97.0,33.0,a,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,2100.0,1000.0,0


Zbiór opisuje kilka wartości fizycznych na aktualnej zmianie w kopalni, a pole *class* zawiera informację o tym czy na następnej zmianie nastąpił groźny wstrząs sejsmiczny, który mógł zakończyć się tąpnięciem. System miałby potencjalnie za zadanie wykrywać takie sygnały ostrzegawcze, aby można było zidentyfikować ryzyko na 8h przed jego wystąpieniem.

## Znaczenie poszczególnych kolumn

* **seismic** - ocena ryzyka zwrócona przez metodę sejsmiczną (a - brak ryzyka, b - małe ryzyko, c - duże ryzyko, d - stan zagrożenia)
* **seismoacoustic** - ocena ryzyka zwrócona przez metodę sejsmoakustyczną (a - brak ryzyka, b - małe ryzyko, c - duże ryzyko, d - stan zagrożenia)
* **shift** - typ aktualnej zmiany (W - wydobycie węgla, N - przygotowawcza);
* **genergy** - energia sejsmiczna zarejestrowana podczas poprzedniej zmiany przez najbardziej aktywny geofon (GMax) z geofonów monitorujących ścianę
* **gpuls** - liczba impulsów zarejestrowanych podczas poprzedniej zmiany przez GMax
* **gdenergy** - odchylenie energii zarejestrowanej przez GMax podczas poprzedniej zmiany od średniej energii zarejestrowanej podczas ośmiu poprzednich zmian
* **gdpuls** - odchylenie liczby impulsów zarejestrowanych przez GMax podczas poprzedniej zmiany od średniej liczby impulsów zarejestrowanych podczas ośmiu poprzednich zmian
* **ghazard** - ocena ryzyka zwrócona przez metodę sejsmoakustyczną na podstawie wyłącznie danych z GMax (a - brak ryzyka, b - małe ryzyko, c - duże ryzyko, d - stan zagrożenia)
* **nbumps** - liczba wstrząsów sejsmicznych zarejestrowanych podczas poprzedniej zmiany
* **nbumps2** - liczba wstrząsów sejsmicznych (w przedziale energii $ [10^2,10^3) $) zarejestrowanych podczas poprzedniej zmiany
* **nbumps3** - liczba wstrząsów sejsmicznych (w przedziale energii $ [10^3,10^4) $) zarejestrowanych podczas poprzedniej zmiany
* **nbumps4** - liczba wstrząsów sejsmicznych (w przedziale energii $ [10^4,10^5) $) zarejestrowanych podczas poprzedniej zmiany
* **nbumps5** - liczba wstrząsów sejsmicznych (w przedziale energii $ [10^5,10^6) $) zarejestrowanych podczas poprzedniej zmiany
* **nbumps6** - liczba wstrząsów sejsmicznych (w przedziale energii $ [10^6,10^7) $) zarejestrowanych podczas poprzedniej zmiany
* **nbumps7** - liczba wstrząsów sejsmicznych (w przedziale energii $ [10^7,10^8) $) zarejestrowanych podczas poprzedniej zmiany
* **nbumps89** - liczba wstrząsów sejsmicznych (w przedziale energii $ [10^8,10^10) $) zarejestrowanych podczas poprzedniej zmiany
* **energy** - łączna energia wstrząsów sejsmicznych zarejestrowanych podczas poprzedniej zmiany
* **maxenergy** - maksymalna energia wstrząsów sejsmicznych zarejestrowanych podczas poprzedniej zmiany
* **class** - 1 oznacza, że podczas następnej zmiany wystąpił wstrząs sejsmiczny o wysokiej energii
("stan niebezpieczny")

In [7]:
seismic_bumps_df.describe()

,genergy,gpuls,gdenergy,gdpuls,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,nbumps6,nbumps7,nbumps89,energy,maxenergy
count,2.584000e+03,2584.000000,2584.000000,2584.000000,2584.000000,2584.000000,2584.000000,2584.000000,2584.000000,2584.0,2584.0,2584.0,2584.000000,2584.000000
mean,9.024252e+04,538.579334,12.375774,4.508901,0.859520,0.393576,0.392802,0.067724,0.004644,0.0,0.0,0.0,4975.270898,4278.850619
std,2.292005e+05,562.652536,80.319051,63.166556,1.364616,0.783772,0.769710,0.279059,0.068001,0.0,0.0,0.0,20450.833222,19357.454882
min,1.000000e+02,2.000000,-96.000000,-96.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
25%,1.166000e+04,190.000000,-37.000000,-36.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
50%,2.548500e+04,379.000000,-6.000000,-6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
75%,5.283250e+04,669.000000,38.000000,30.250000,1.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0,2600.000000,2000.000000
max,2.595650e+06,4518.000000,1245.000000,838.000000,9.000000,8.000000,7.000000,3.000000,1.000000,0.0,0.0,0.0,402000.000000,400000.000000


Kolumny nbumbs6,7,89 nie zawierają niezerowych wartości, więc możemy się ich od razu pozbyć.

In [8]:
seismic_bumps_df.drop(columns=['nbumps6', "nbumps7", "nbumps89"], inplace=True)

In [12]:
seismic_bumps_df.drop(columns=["seismic", "seismoacoustic", "shift", "ghazard"]).corr().style.background_gradient("Wistia")

,genergy,gpuls,gdenergy,gdpuls,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,energy,maxenergy,class
genergy,1.000000,0.748020,0.048514,0.071554,0.220720,0.143587,0.191753,0.150589,-0.009863,0.080828,0.064405,0.128160
gpuls,0.748020,1.000000,0.293036,0.382906,0.300923,0.207390,0.225695,0.256547,0.049450,0.187350,0.164263,0.201042
gdenergy,0.048514,0.293036,1.000000,0.811944,0.030039,0.041246,-0.012189,0.036916,0.123229,0.105971,0.108572,0.002957
gdpuls,0.071554,0.382906,0.811944,1.000000,0.057996,0.051106,0.014735,0.066195,0.141044,0.143277,0.143646,0.021788
nbumps,0.220720,0.300923,0.030039,0.057996,1.000000,0.804978,0.803364,0.395052,0.069613,0.347852,0.273714,0.248144
nbumps2,0.143587,0.207390,0.041246,0.051106,0.804978,1.000000,0.350724,0.161296,-0.005251,0.124650,0.085032,0.201381
nbumps3,0.191753,0.225695,-0.012189,0.014735,0.803364,0.350724,1.000000,0.175301,0.046498,0.244083,0.176480,0.205327
nbumps4,0.150589,0.256547,0.036916,0.066195,0.395052,0.161296,0.175301,1.000000,-0.016580,0.489828,0.416483,0.075458
nbumps5,-0.009863,0.049450,0.123229,0.141044,0.069613,-0.005251,0.046498,-0.016580,1.000000,0.773605,0.808406,0.004834
energy,0.080828,0.187350,0.105971,0.143277,0.347852,0.124650,0.244083,0.489828,0.773605,1.000000,0.989547,0.058431


Jako że część zmiennych jest kategoryczna, musimy przeprowadzić ich konwersję do typów numerycznych, np. za pomocą one-hot encoding.

In [13]:
class_namme = seismic_bumps_df["class"]
seismic_bumps_df = pd.get_dummies(seismic_bumps_df.drop(columns="class"))
seismic_bumps_df.sample(10)

,genergy,gpuls,gdenergy,gdpuls,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,energy,...,seismic_a,seismic_b,seismoacoustic_a,seismoacoustic_b,seismoacoustic_c,shift_N,shift_W,ghazard_a,ghazard_b,ghazard_c
994,2870.0,22.0,-40.0,-45.0,0.0,0.0,0.0,0.0,0.0,0.0,...,True,False,True,False,False,True,False,True,False,False
1733,27390.0,674.0,-17.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,True,True,False,False,False,True,True,False,False
1885,64920.0,690.0,103.0,7.0,1.0,0.0,1.0,0.0,0.0,1000.0,...,True,False,True,False,False,False,True,True,False,False
2014,18490.0,451.0,-34.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,True,False,True,False,False,True,True,False,False
468,104410.0,631.0,54.0,50.0,3.0,2.0,1.0,0.0,0.0,1600.0,...,True,False,False,True,False,False,True,True,False,False
780,41740.0,331.0,37.0,29.0,2.0,2.0,0.0,0.0,0.0,1100.0,...,True,False,True,False,False,False,True,True,False,False
1929,13640.0,210.0,123.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,...,True,False,True,False,False,True,False,True,False,False
1324,36230.0,822.0,177.0,151.0,1.0,0.0,1.0,0.0,0.0,8000.0,...,True,False,False,True,False,False,True,False,True,False
1709,21350.0,376.0,-38.0,-31.0,0.0,0.0,0.0,0.0,0.0,0.0,...,True,False,False,True,False,False,True,True,False,False
1853,4070.0,173.0,-61.0,-54.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,True,False,True,False,True,False,False,True,False


Okazuje się, że dla danego okresu żaden z czujników nigdy nie zwrócił wartości "d" oznaczającej stan zagrożenia. Na koniec zobaczmy jeszcze jak rozkładają się nasze klasy, ponieważ to ich będziemy używać jako wyznaczniki tego czy dana obserwacja jest anomalią.

In [15]:
class_namme.value_counts()

class
0    2414
1     170
Name: count, dtype: int64

In [16]:
seismic_bumps_df["class"] = class_namme
seismic_bumps_df.to_parquet("../data/seismic_bumps_df.parquet")